https://qiita.com/cvusk/items/1f3b178f34c39beb29ff#_reference-3c5df6e5237c0c705690
https://github.com/shibuiwilliam/keras_opt/blob/master/tpe_nn.ipynb


In [1]:
from hyperopt import hp, tpe, Trials, fmin
import numpy as np
import pandas as pds
import random
from keras.layers import Activation, Dropout, BatchNormalization, Dense
from keras.models import Sequential
from keras.datasets import mnist
from keras.metrics import categorical_crossentropy
from keras.utils import np_utils
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

/home/yuki/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# MNIST class
class MNIST():
    def __init__(self,
                 l1_out=512, 
                 l2_out=512, 
                 l1_drop=0.2, 
                 l2_drop=0.2, 
                 bn1=0,
                 bn2=0,
                 batch_size=100, 
                 epochs=10, 
                 validation_split=0.1):
        self.l1_out = l1_out
        self.l2_out = l2_out
        self.l1_drop = l1_drop
        self.l2_drop = l2_drop
        self.bn1 = bn1
        self.bn2 = bn2
        self.batch_size = batch_size
        self.epochs = epochs
        self.validation_split = validation_split
        self.__x_train, self.__x_test, self.__y_train, self.__y_test = self.mnist_data()
        self.__model = self.mnist_model()
        params = """
        validation_split:\t{0}
        l1_drop:\t{1}
        l2_drop:\t{2}
        l1_out:\t{3}
        l2_out:\t{4}
        bn1:\t{5}
        bn2:\t{6}
        batch_size:\t{7}
        epochs:\t{8}
        """.format(self.validation_split,
                   self.l1_drop, self.l2_drop,
                   self.l1_out, self.l2_out,
                   self.bn1, self.bn2,
                   self.batch_size, self.epochs)
        print(params)
        
    # load mnist data from keras dataset
    def mnist_data(self):
        (X_train, y_train), (X_test, y_test) = mnist.load_data()
        X_train = X_train.reshape(60000, 784)
        X_test = X_test.reshape(10000, 784)

        X_train = X_train.astype('float32')
        X_test = X_test.astype('float32')
        X_train /= 255
        X_test /= 255

        Y_train = np_utils.to_categorical(y_train, 10)
        Y_test = np_utils.to_categorical(y_test, 10)
        return X_train, X_test, Y_train, Y_test
    
    # mnist model
    def mnist_model(self):
        model = Sequential()
        model.add(Dense(self.l1_out, input_shape=(784,)))
        if self.bn1 == 0:
            model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Dropout(self.l1_drop))
        model.add(Dense(self.l2_out))
        if self.bn2 == 0:
            model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Dropout(self.l2_drop))
        model.add(Dense(10))
        model.add(Activation('softmax'))
        model.compile(loss='categorical_crossentropy',
                      optimizer=Adam(),
                      metrics=['accuracy'])

        return model
    
    # fit mnist model
    def mnist_fit(self):
        early_stopping = EarlyStopping(patience=0, verbose=1)
        
        self.__model.fit(self.__x_train, self.__y_train,
                       batch_size=self.batch_size,
                       epochs=self.epochs,
                       verbose=0,
                       validation_split=self.validation_split,
                       callbacks=[early_stopping])
    
    # evaluate mnist model
    def mnist_evaluate(self):
        self.mnist_fit()
        
        evaluation = self.__model.evaluate(self.__x_test, self.__y_test, batch_size=self.batch_size, verbose=0)
        return evaluation

In [3]:
# function to run mnist class
def run_mnist(args):
    _mnist = MNIST(**args)
    mnist_evaluation = _mnist.mnist_evaluate()
    print("loss:{0} \t\t accuracy:{1}".format(mnist_evaluation[0], mnist_evaluation[1]))
    return mnist_evaluation[0]

In [4]:
hyperopt_parameters = {
    'validation_split': hp.uniform('validation_split', 0.0, 0.3),
    'l1_drop': hp.uniform('l1_drop', 0.0, 0.3),
    'l2_drop': hp.uniform('l2_drop', 0.0, 0.3),
    'l1_out': hp.choice('l1_out', [64, 128, 256, 512, 1024]),
    'l2_out': hp.choice('l2_out', [64, 128, 256, 512, 1024]),
    'bn1': hp.choice('bn1', [0, 1]),
    'bn2': hp.choice('bn2', [0, 1]),
    'batch_size': hp.choice('batch_size', [10, 100, 500]),
    'epochs': hp.choice('epochs', [5, 10, 20]),
}

In [5]:
max_evals = 20

trials = Trials()

best = fmin(run_mnist, hyperopt_parameters, 
            algo=tpe.suggest, max_evals=max_evals,
           trials=trials, verbose=1)


        validation_split:	0.09863089067206789
        l1_drop:	0.06553750322108096
        l2_drop:	0.19207860503895807
        l1_out:	128
        l2_out:	128
        bn1:	0
        bn2:	0
        batch_size:	500
        epochs:	20
        
Epoch 00009: early stopping
loss:0.07320360955782235 		 accuracy:0.9771000057458877

        validation_split:	0.07619821481480546
        l1_drop:	0.17358170348662957
        l2_drop:	0.2043092008548389
        l1_out:	256
        l2_out:	1024
        bn1:	0
        bn2:	0
        batch_size:	10
        epochs:	10
        
Epoch 00007: early stopping
loss:0.06817380543545187 		 accuracy:0.9779999960064888

        validation_split:	0.2745822189197116
        l1_drop:	0.04121886544239229
        l2_drop:	0.11181034230077849
        l1_out:	128
        l2_out:	128
        bn1:	0
        bn2:	0
        batch_size:	10
        epochs:	20
        
Epoch 00006: early stopping
loss:0.08124729674684932 		 accuracy:0.9749999961256981

        validation_sp

In [6]:
print(best)

{'batch_size': 0, 'bn1': 1, 'bn2': 0, 'epochs': 1, 'l1_drop': 0.27446756661724, 'l1_out': 3, 'l2_drop': 0.11233450376312329, 'l2_out': 2, 'validation_split': 0.11562556825542107}
